In [1]:
from selenium.webdriver.common.by import By
import selenium as sel
import pandas as pd
import time
from datetime import datetime

import os
import sys
cwd = os.getcwd()
cwd = cwd.replace("Notebooks", "Scripts")
sys.path.insert(0, cwd.replace("\\\\", "\\"))
import base as bs

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException


In [2]:
#Empresa con la cual vamos a extraer los articulos
empresa = input("Digite la empresa a extraer: ").lower().strip()
if " " in empresa:
    empresa = empresa.replace(" ", "%20")
revista = "laSillaVacia"

In [3]:
# crear driver... MODIFICAR DEPENDIENDO DEL NAVEGADOR
driver = bs.ejecutar_driver(f'https://www.lasillavacia.com/buscar?q={empresa}&cat=all')

i = 1
try:
    while i < 2:
        driver.implicitly_wait(10)
        button = driver.find_element(By.XPATH, './/a[contains(@class, "load-more-results d-block p xl black-3d bold uppercase text-center")]')
        button.click()
except:
    pass

In [4]:
# "c c-d _g _g-md c-m-l c--m-n" Contiene tema
articulos = driver.find_elements(By.XPATH, './/article')

In [5]:
titulares = []
for art in articulos:
    url = bs.get_url_sv(art)
    if url == None:
        continue
    if not(bs.existedb(url, "database")):
        titulo = bs.get_titulo_sv(art)
        titulares.append({'Fecha Extraccion':datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                        'Titulo': titulo,
                        'URL': url,
                        'Empresa': empresa
                        })

In [6]:
from tqdm import tqdm

In [7]:
for tit in tqdm(titulares[:10]):
    driver.get(tit["URL"])
    driver.implicitly_wait(10)
    tit["Autor"] = bs.get_autor_sv(driver)
    tit["Fecha Publicacion"] = bs.get_fecha_sv(driver)
    tit["Imagen"] = bs.get_imag_sv(driver)
    # tit["Resumen"] = get_resumen(driver)
    tit["Fuente"] = "La Silla Vacía"
    tit["Contenido"] = bs.get_contenido_sv(driver)

 10%|█         | 1/10 [00:07<01:11,  8.00s/it]

2023-04-12 00:00:00
2023-04-12 00:00:00


 30%|███       | 3/10 [00:26<01:00,  8.65s/it]

2023-03-30 00:00:00
2023-02-07 00:00:00


 40%|████      | 4/10 [00:30<00:41,  6.86s/it]

In [ ]:
df = pd.DataFrame(titulares)
# bs.writeData("database", df)

In [ ]:
driver.close()

In [ ]:
df.head()

,Fecha Extraccion,Titulo,URL,Empresa,Autor,Fecha Publicacion,Imagen,Fuente,Contenido
